In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib
from matplotlib.ticker import MaxNLocator
from collections import namedtuple
import time
style.use('tableau-colorblind10')
import matplotlib.colors as mcolors
import cv2
import os
import shutil
###########################################################################################################################################################



def plot_gen_script(plot_dataframe,scores,cohort_size,colors,plot_names,dir_loc):
        fig, ax1 = plt.subplots(figsize=(16, 9))
        fig.text(.80, 0.25, 'Day {}'.format(plot_dataframe.year),
             fontsize=65, color='gray',
             ha='right', va='bottom', alpha=0.5)
        pos = np.arange(len(testNames))
        rects = ax1.barh(pos, [scores[k].rank for k in testNames],
                         align='center',
                         height=0.5,
                         tick_label=testNames, color=colors)

        ax1.set_title("Top 10 countries by number of confirmed cases".format(plot_names[-1],plot_names[-2],plot_names[-3])
                     ,fontsize=22,pad=30,color= "black", weight="bold")
                     
                     
        ax1.set_xlim([0, cohort_size*1.2])
        ax1.xaxis.set_major_locator(MaxNLocator(11))
        for tick in ax1.yaxis.get_major_ticks():
                tick.label.set_fontsize(17) 
        for tick in ax1.xaxis.get_major_ticks():
                tick.label.set_fontsize(17) 
        rect_labels = []
        for rect in rects:
            width = int(rect.get_width())
            rankStr = width
            if width < cohort_size:
                xloc = 5
                clr = 'black'
                align = 'left'
            else:
                xloc = 5
                clr = 'black'
                align = 'left'

            yloc = rect.get_y() + rect.get_height() / 2
            label = ax1.annotate(rankStr, xy=(width, yloc), xytext=(xloc, 0),
                                textcoords="offset points",
                                ha=align, va='center',
                                color=clr, clip_on=True)
            rect_labels.append(label)
        plt.savefig(dir_loc+"/"+str(i)+".png")
        plt.close('all')

In [2]:
root_dir= os.getcwd()
image_folder = root_dir+"/image_folder"
GDP_data_loc = root_dir+"/confirmed_top_10.xlsx"

# per iteration Frame's you wnat to keep in your video
interpolation_limit = 5

# How many top numbers you want to show
top_show_numbers = 10

### converting to per 10 million range 
divider= 1

base_data=pd.read_excel(GDP_data_loc,index_col=[0])
for col in base_data.columns:
    base_data[col] = pd.to_numeric(base_data[col], errors='coerce')

# GLOBAL CONSTANTS
data_vars= namedtuple('GDP', ['year'])
Score = namedtuple('Score', ['rank'])
base_data=base_data.T.bfill().ffill().T

In [3]:
        df1 = base_data
        ########################################################################################################################
        
        ##### Generating color list for ploting ###
        colours=['midnightblue','blue','royalblue','mediumslateblue','cornflowerblue','darkviolet','mediumorchid','violet','lightsteelblue','deepskyblue']
        ### converting to per 10 million range 
        df1=df1 / divider
        ##########################################################################################################################
        #### generating interpolated datasets for more ganurlarity
        md=pd.DataFrame(columns=['Year']+list(df1.T.columns))

        for i in range(df1.T.shape[0]-1):
            for x in range(interpolation_limit):
                if x==0:
                    append_data = dict(zip(['Year']+list(df1.T.columns),np.append(df1.T.index[i],df1.T.iloc[i,:].values)))

                else:
                    append_data = dict(zip(['Year']+list(df1.T.columns),np.append(df1.T.index[i],np.full(md.shape[1]-1, np.nan))))
                md=md.append(append_data,ignore_index=True) 

        ### converting to numerical columns as the data still will be in object form 
        for col in md.columns:
            if col != "Year":
                md[col] = pd.to_numeric(md[col], errors='coerce')

        # Linear interpolatio for generating data on more gnallure label
        md=md.interpolate(method='linear')
        md.index=md.Year
        md=md.drop(['Year'],1)
        df1=md.T
        ########################################################################################################################
        plot_names = list(md.columns)
        print(plot_names)
        colour_dict=dict(zip(df1.index,colours[:len(df1.index)]))

        for i in range(df1.shape[1]):
            print("Genearting plot >>>>>>>>>>  {}".format(i))
            testNames = list((df1.iloc[:,i].sort_values()).index)
            selected_data_vars= data_vars(df1.columns[i])
            
            ## giving the index values here
            scores = dict(zip(testNames,
                              (Score(p) for p in df1.iloc[:,i].sort_values().values)))

            ## getting the respective colours of the index here
            colours =[colour_dict[i] for i in list((df1.iloc[:,i].sort_values()).index)]

            ## giving the max value of index for that time here
            cohort_size =  max(df1.iloc[:,i].values)

            plot_gen_script(selected_data_vars,scores,cohort_size,colours,plot_names,image_folder)

        print("image generation completed")
        ########################################################################################################################
        print("generating video out of images")
        testNames = list((df1.iloc[:,1].sort_values()).index)
        video_name = 'animated bar chart.avi'.format(testNames[-1],testNames[-2],testNames[-3])


        image = [img for img in os.listdir(image_folder) if img.endswith(".png")]

        images=[str(i)+".png" for i in range(len(image))]

        frame = cv2.imread(os.path.join(image_folder, images[0]))
        height, width, layers = frame.shape
        video = cv2.VideoWriter(video_name, 0, 10, (width,height))
        for image in images:
            video.write(cv2.imread(os.path.join(image_folder, image)))
        cv2.destroyAllWindows()
        video.release()

['US', 'Spain', 'Italy', 'France', 'Germany', 'United Kingdom', 'Turkey', 'Iran', 'China', 'Russia']
Genearting plot >>>>>>>>>>  0
Genearting plot >>>>>>>>>>  1
Genearting plot >>>>>>>>>>  2
Genearting plot >>>>>>>>>>  3
Genearting plot >>>>>>>>>>  4
Genearting plot >>>>>>>>>>  5
Genearting plot >>>>>>>>>>  6
Genearting plot >>>>>>>>>>  7
Genearting plot >>>>>>>>>>  8
Genearting plot >>>>>>>>>>  9
Genearting plot >>>>>>>>>>  10
Genearting plot >>>>>>>>>>  11
Genearting plot >>>>>>>>>>  12
Genearting plot >>>>>>>>>>  13
Genearting plot >>>>>>>>>>  14
Genearting plot >>>>>>>>>>  15
Genearting plot >>>>>>>>>>  16
Genearting plot >>>>>>>>>>  17
Genearting plot >>>>>>>>>>  18
Genearting plot >>>>>>>>>>  19
Genearting plot >>>>>>>>>>  20
Genearting plot >>>>>>>>>>  21
Genearting plot >>>>>>>>>>  22
Genearting plot >>>>>>>>>>  23
Genearting plot >>>>>>>>>>  24
Genearting plot >>>>>>>>>>  25
Genearting plot >>>>>>>>>>  26
Genearting plot >>>>>>>>>>  27
Genearting plot >>>>>>>>>>  28
Genearting

Genearting plot >>>>>>>>>>  257
Genearting plot >>>>>>>>>>  258
Genearting plot >>>>>>>>>>  259
Genearting plot >>>>>>>>>>  260
Genearting plot >>>>>>>>>>  261
Genearting plot >>>>>>>>>>  262
Genearting plot >>>>>>>>>>  263
Genearting plot >>>>>>>>>>  264
Genearting plot >>>>>>>>>>  265
Genearting plot >>>>>>>>>>  266
Genearting plot >>>>>>>>>>  267
Genearting plot >>>>>>>>>>  268
Genearting plot >>>>>>>>>>  269
Genearting plot >>>>>>>>>>  270
Genearting plot >>>>>>>>>>  271
Genearting plot >>>>>>>>>>  272
Genearting plot >>>>>>>>>>  273
Genearting plot >>>>>>>>>>  274
Genearting plot >>>>>>>>>>  275
Genearting plot >>>>>>>>>>  276
Genearting plot >>>>>>>>>>  277
Genearting plot >>>>>>>>>>  278
Genearting plot >>>>>>>>>>  279
Genearting plot >>>>>>>>>>  280
Genearting plot >>>>>>>>>>  281
Genearting plot >>>>>>>>>>  282
Genearting plot >>>>>>>>>>  283
Genearting plot >>>>>>>>>>  284
Genearting plot >>>>>>>>>>  285
Genearting plot >>>>>>>>>>  286
Genearting plot >>>>>>>>>>  287
Genearti